In [14]:
import tkinter as tk
import heapq

ROWS = 7  # Number of rows
COLS = 7  # Number of columns
CELL_SIZE = 100
DEADLOCK_THRESHOLD = 2  # Number of steps before resolving deadlock

# A* Algorithm to find the shortest path
def a_star(grid, start, goal, obstacles):
    def heuristic(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    rows, cols = len(grid), len(grid[0])
    open_list = []
    heapq.heappush(open_list, (0, start))

    came_from = {}
    g_score = {start: 0}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)

            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and grid[neighbor[0]][neighbor[1]] != 'X' and neighbor not in obstacles:
                tentative_g_score = g_score[current] + 1
                if tentative_g_score < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score, neighbor))

    return None  # No path found

# Grid and start/goal positions
def create_grid():
    return [['.' for _ in range(COLS)] for _ in range(ROWS)]

class BotSimulator(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Warehouse Bot Simulator")
        self.geometry(f"{COLS * CELL_SIZE}x{ROWS * CELL_SIZE + 200}")
        self.canvas = tk.Canvas(self, width=COLS * CELL_SIZE, height=ROWS * CELL_SIZE)
        self.canvas.pack()

        # Start and reset buttons
        self.start_button = tk.Button(self, text="Start", command=self.start_simulation)
        self.start_button.pack()
        self.reset_button = tk.Button(self, text="Reset", command=self.reset_simulation)
        self.reset_button.pack()

        # Performance metrics display
        self.metrics_label = tk.Label(self, text="")
        self.metrics_label.pack()

        # Initialize grid and bot information
        self.grid = create_grid()
        self.bot_positions = [(0, 0), (3, 0),(4,0),(5,0)]  # Start positions for 2 bots
        self.goals = [(0, 6), (6, 6),(4,6),(5,6)]  # Goal positions for 2 bots
        self.obstacles = [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1)]  # Fixed obstacles
        self.bot_paths = [[], []]  # Paths for the bots
        self.total_time = [0, 0]  # Time taken by each bot
        self.total_commands = [0, 0]  # Commands issued by each bot
        self.waiting_time = [0, 0]  # Track waiting time for each bot
        self.deadlock_counter = [0, 0]  # Deadlock counter to track how long each bot has been waiting
        self.waiting = [False, False]  # Track whether a bot is waiting due to collision

        # Draw the initial grid
        self.draw_grid()

        # Flag to control the simulation loop
        self.simulation_running = False

        # Bind mouse click to add/remove obstacles
        self.canvas.bind("<Button-1>", self.toggle_obstacle)

    def start_simulation(self):
        self.start_button.config(state="disabled")
        self.simulation_running = True

        # Calculate the paths for all bots using A*
        for i in range(2):  # Adjusted for 2 bots
            path = a_star(self.grid, self.bot_positions[i], self.goals[i], self.obstacles)
            self.bot_paths[i] = path if path else []

        # Start simulation loop
        self.step()

    def draw_grid(self):
        self.canvas.delete("all")

        # Draw the cells
        for row in range(ROWS):
            for col in range(COLS):
                x1, y1 = col * CELL_SIZE, row * CELL_SIZE
                x2, y2 = x1 + CELL_SIZE, y1 + CELL_SIZE
                self.canvas.create_rectangle(x1, y1, x2, y2, fill="white")

        # Draw obstacles
        for ox, oy in self.obstacles:
            self.draw_cell(ox, oy, "black")

        # Draw goals
        for idx, goal in enumerate(self.goals):
            self.draw_cell(goal[0], goal[1], "green", text=f"G{idx + 1}")

        # Draw bots
        colors = ["blue", "red"]
        for idx, position in enumerate(self.bot_positions):
            self.draw_cell(position[0], position[1], colors[idx], text=f"A{idx + 1}")

    def draw_cell(self, row, col, color, text=None):
        x1, y1 = col * CELL_SIZE, row * CELL_SIZE
        x2, y2 = x1 + CELL_SIZE, y1 + CELL_SIZE

        self.canvas.create_rectangle(x1, y1, x2, y2, fill=color)
        if text:
            self.canvas.create_text(x1 + CELL_SIZE // 2, y1 + CELL_SIZE // 2, text=text, font=("Helvetica", 18), fill="white")

    def step(self):
        if not self.simulation_running:
            return

        self.update_bots()
        self.draw_grid()

        # Continue the simulation if the bots haven't reached their goals
        if any(self.bot_positions[i] != self.goals[i] for i in range(2)):  # Adjusted for 2 bots
            self.after(500, self.step)  # Run the step function again after 500 ms
        else:
            print("All bots have reached their goals!")
            self.show_performance_metrics()

    def update_bots(self):
        for i in range(2):  # Adjusted for 2 bots
            if self.bot_positions[i] != self.goals[i] and self.bot_paths[i]:
                next_pos = self.bot_paths[i][0]

                # Check for potential collision with the other bot
                other_bot = 1 - i  # Get the index of the other bot
                if next_pos == self.bot_positions[other_bot]:
                    # Collision detected, mark this bot as waiting
                    self.waiting[i] = True
                    self.deadlock_counter[i] += 1

                    # Check for deadlock (both bots waiting)
                    if self.deadlock_counter[i] >= DEADLOCK_THRESHOLD:
                        print(f"Deadlock detected for bot A{i+1}, resolving by letting it move.")
                        self.waiting[i] = False  # Force this bot to move
                        self.deadlock_counter[i] = 0  # Reset the deadlock counter
                    continue  # Skip moving this bot due to collision
                else:
                    # No collision, reset waiting status and move the bot
                    self.waiting[i] = False
                    self.bot_positions[i] = next_pos
                    self.bot_paths[i] = self.bot_paths[i][1:]
                    self.total_time[i] += 1

    def show_performance_metrics(self):
        metrics = "Bot Performance Metrics:\n"
        for i in range(2):  # Adjusted for 2 bots
            metrics += f"Bot A{i + 1}: Time = {self.total_time[i]}, Commands = {self.total_commands[i]}\n"
        self.metrics_label.config(text=metrics)

    def reset_simulation(self):
        self.start_button.config(state="normal")
        self.simulation_running = False
        self.bot_positions = [(0, 0), (5, 0)]  # Adjusted for 2 bots
        self.bot_paths = [[], []]
        self.total_time = [0, 0]
        self.total_commands = [0, 0]
        self.waiting_time = [0, 0]
        self.deadlock_counter = [0, 0]
        self.grid = create_grid()
        self.draw_grid()

    def toggle_obstacle(self, event):
        col = event.x // CELL_SIZE
        row = event.y // CELL_SIZE
        if (row, col) in self.obstacles:
            self.obstacles.remove((row, col))
        else:
            self.obstacles.append((row, col))
        self.draw_grid()


if __name__ == "__main__":
    app = BotSimulator()
    app.mainloop()
       


Deadlock detected for bot A1, resolving by letting it move.
Deadlock detected for bot A1, resolving by letting it move.
Deadlock detected for bot A1, resolving by letting it move.
Deadlock detected for bot A1, resolving by letting it move.
Deadlock detected for bot A1, resolving by letting it move.


In [15]:
class BotSimulator(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Warehouse Bot Simulator")
        self.geometry(f"{COLS * CELL_SIZE}x{ROWS * CELL_SIZE + 200}")
        self.canvas = tk.Canvas(self, width=COLS * CELL_SIZE, height=ROWS * CELL_SIZE)
        self.canvas.pack()

        # Start and reset buttons
        self.start_button = tk.Button(self, text="Start", command=self.start_simulation)
        self.start_button.pack()
        self.reset_button = tk.Button(self, text="Reset", command=self.reset_simulation)
        self.reset_button.pack()

        # Performance metrics display
        self.metrics_label = tk.Label(self, text="")
        self.metrics_label.pack()

        # Initialize grid and bot information
        self.grid = create_grid()
        self.bot_positions = [(0, 0), (2, 0)]  # Start positions for 2 bots
        self.goals = [(3, 0), (1, 0)]  # Goal positions for 2 bots
        self.obstacles = [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1)]  # Fixed obstacles
        self.bot_paths = [[], []]  # Paths for the bots
        self.total_time = [0, 0]  # Time taken by each bot
        self.total_commands = [0, 0]  # Commands issued by each bot
        self.waiting_time = [0, 0]  # Track waiting time for each bot
        self.deadlock_counter = [0, 0]  # Deadlock counter to track how long each bot has been waiting
        self.waiting = [False, False]  # Track whether a bot is waiting due to collision

        # Step counter to detect impossible cases
        self.step_count = 0  # Track total steps taken in the simulation
        self.max_steps = 25  # Threshold to detect impossible case

        # Draw the initial grid
        self.draw_grid()

        # Flag to control the simulation loop
        self.simulation_running = False

        # Bind mouse click to add/remove obstacles
        self.canvas.bind("<Button-1>", self.toggle_obstacle)

    def start_simulation(self):
        self.start_button.config(state="disabled")
        self.simulation_running = True

        # Calculate the paths for all bots using A*
        for i in range(2):  # Adjusted for 2 bots
            path = a_star(self.grid, self.bot_positions[i], self.goals[i], self.obstacles)
            self.bot_paths[i] = path if path else []

        # Start simulation loop
        self.step()

    def step(self):
        if not self.simulation_running:
            return

        self.update_bots()
        self.draw_grid()

        self.step_count += 1  # Increment step count

        # Check if the step count exceeds the max steps (impossible case)
        if self.step_count > self.max_steps:
            self.simulation_running = False
            print("Impossible case detected after 25 commands!")
            self.metrics_label.config(text="Impossible case detected after 25 commands!")
            return

        # Continue the simulation if the bots haven't reached their goals
        if any(self.bot_positions[i] != self.goals[i] for i in range(2)):  # Adjusted for 2 bots
            self.after(500, self.step)  # Run the step function again after 500 ms
        else:
            print("All bots have reached their goals!")
            self.show_performance_metrics()

    def update_bots(self):
        # Adjusted to move Bot A2 (index 1) first, then Bot A1 (index 0)
        for i in [1, 0]:  # Prioritize Bot A2 (index 1) over Bot A1 (index 0)
            if self.bot_positions[i] != self.goals[i] and self.bot_paths[i]:
                next_pos = self.bot_paths[i][0]

                # Check for potential collision with the other bot
                other_bot = 1 - i  # Get the index of the other bot
                if next_pos == self.bot_positions[other_bot]:
                    # Collision detected, mark this bot as waiting
                    self.waiting[i] = True
                    self.deadlock_counter[i] += 1

                    # Check for deadlock (both bots waiting)
                    if self.deadlock_counter[i] >= DEADLOCK_THRESHOLD:
                        print(f"Deadlock detected for bot A{i+1}, resolving by letting it move.")
                        self.waiting[i] = False  # Force this bot to move
                        self.deadlock_counter[i] = 0  # Reset the deadlock counter
                    continue  # Skip moving this bot due to collision
                else:
                    # No collision, reset waiting status and move the bot
                    self.waiting[i] = False
                    self.bot_positions[i] = next_pos
                    self.bot_paths[i] = self.bot_paths[i][1:]
                    self.total_time[i] += 1
                    self.total_commands[i] += 1  # Increment command count

    def show_performance_metrics(self):
        # Calculate average and maximum commands issued
        total_commands = sum(self.total_commands)
        num_bots = len(self.total_commands)
        avg_commands = total_commands / num_bots
        max_commands = max(self.total_commands)

        # Display metrics
        metrics = f"Bot Performance Metrics:\n"
        for i in range(2):  # Adjusted for 2 bots
            metrics += f"Bot A{i + 1}: Commands = {self.total_commands[i]}\n"
        metrics += f"Average Commands: {avg_commands:.2f}\n"
        metrics += f"Maximum Commands: {max_commands}\n"
        self.metrics_label.config(text=metrics)

    def reset_simulation(self):
        self.start_button.config(state="normal")
        self.simulation_running = False
        self.bot_positions = [(0, 0), (2, 0)] # Adjusted for 2 bots
        self.bot_paths = [[], []]
        self.total_time = [0, 0]
        self.total_commands = [0, 0]
        self.waiting_time = [0, 0]
        self.deadlock_counter = [0, 0]
        self.step_count = 0  # Reset step count for new simulation
        self.grid = create_grid()
        self.draw_grid()

    def toggle_obstacle(self, event):
        col = event.x // CELL_SIZE
        row = event.y // CELL_SIZE
        if (row, col) in self.obstacles:
            self.obstacles.remove((row, col))
        else:
            self.obstacles.append((row, col))
        self.draw_grid()



In [17]:
# Create an instance of the BotSimulator
bot_simulator = BotSimulator()

# Start the Tkinter main loop
bot_simulator.mainloop()


AttributeError: '_tkinter.tkapp' object has no attribute 'draw_grid'